<a href="https://colab.research.google.com/drive/1NsRNRf4SzQejjy8d04LM_AQufaqezFSg" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install VectorVault

In [ ]:
!pip install vector-vault 

<br>
<br>

## Get API key to access the Vault Cloud

Enter your name, email, and make a password below to get an API key.

In [ ]:
from vectorvault import register

register(first_name='John', last_name='Smith', email='john@smith.com', password='make_a_password')

<br>

# Connect to VectorVault
In the following code, we import the main Vault class to connect to a vault as well as set our OpenAI API key as an environment varible. We will need that set to process the vectors with `get_vectors` and use the `get_chat()` function later.

In [ ]:
%env OPENAI_API_KEY = YOUR_OPENAI_API_KEY

In [ ]:
from vectorvault import Vault, download_url

vault = Vault(user='john@smith.com', api_key='YOUR_VECTOR_VAULT_API_KEY', vault='philosophy', verbose=True)


<br>

# Download the Philosophy Books 
1. Download the book texts from the handpicked, refinded data repo of no-copywrite philosophy texts
2. Get vectors 
3. Add to Vault

This is a large amount of text and will take roughly 9min to get the vectors and then save them to the cloud. After this is done, referencing the all of them at once will take less than a second.

*A Free account will not have the resources to fully finish this. You can stop after the first couple books, or get a [Personal account](https://buy.stripe.com/7sI03AgMldS5b7y7st)*

In [ ]:
import re
import requests 

vault.delete()

# Get the books from the repo
response = requests.get("https://api.github.com/repos/John-Rood/Philosophy/contents/books")
files = response.json()

# Make a loop to download each book since there are 37
for file in files:
  if file["name"].endswith(".txt"):

    # Download the book text
    file_url = file["download_url"]
    text = download_url(file_url)
    print(f"Processing {file_url}")

    # Extract metadata
    title = re.search(r"Title:\s*(.+)", text)
    author = re.search(r"Author:\s*(.+)", text)

    # Handle formatting issues
    title = title.group(1).strip() if title else "Unknown Title"
    author = author.group(1).strip() if author else "Unknown Author"

    meta = {
              "title": title,
              "author": author,
            }

    # Add for processing
    vault.add(text, meta)

    # Get vectors for all data added
    vault.get_vectors()
    
    # Save data with vectors to the Vault Cloud
    vault.save()


# Ask Your Question

In [ ]:
from vectorvault import wrap

user_input = input("What's your next question?: ")

# Get response from Language model
answer = vault.get_chat(user_input, get_context=True)

print("Question:", f"\n{wrap(user_input)}", "\n\nAnswer:", f"\n{wrap(answer)}")


<br>

## See where the answer is coming from with `return_context=True`
In this response, we add conversation history to to the "get_chat()" function. We also print the source context that the llm uses for the response. (Since the sources are returned along with the response, it comes in the form of a dictionary)

In [ ]:
user_input2 = input("What's your next question?: ")

history = user_input + answer

# Get response from Language model
vault_response = vault.get_chat(user_input2, history=history, get_context=True, return_context=True)

# Since 'return_context=True', the response is returned in a dictionary
answer = vault_response['response']
print("Question:", f"\n{wrap(user_input2)}", "\n\nAnswer:", f"\n{wrap(answer)}")

# show the context used to generate the answer
for item in vault_response['context']:
    print("\n\n", f"Title: {item['metadata']['title']}")
    print(f"Author: {item['metadata']['author']}","\n\n")
    print(wrap(item['data']))


In [ ]:
user_input3 = input("What's your next question?: ")

history = history + user_input2 + answer

# Get response from Language model
response = vault.get_chat(user_input3, history=history, get_context=True)

print("Question:", f"\n{wrap(user_input3)}", "\n\nAnswer:", f"\n{wrap(response)}")
